# 1.- Realizar todos los ejercicios propuestos de libro

In [1]:
import org.apache.log4j.Level
import org.apache.spark.sql.{DataFrame, SaveMode, SparkSession}

val spark = SparkSession
 .builder()
 .appName("Capitulo2")
 .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://L2202039.bosonit.local:4042
SparkContext available as 'sc' (version = 3.1.3, master = local[*], app id = local-1652175854574)
SparkSession available as 'spark'


import org.apache.log4j.Level
import org.apache.spark.sql.{DataFrame, SaveMode, SparkSession}
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@228d301b


In [4]:
// Get the M&M data set filename
val mnmFile = "C:/Users/alejandro.perez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/iot-devices/iot_devices.json"



// Read the file into a Spark DataFrame
val df = spark.read.format("json")
 .load(mnmFile)
// Aggregate counts of all colors and groupBy() State and Color
// orderBy() in descending order

mnmFile: String = C:/Users/alejandro.perez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/iot-devices/iot_devices.json
df: org.apache.spark.sql.DataFrame = [battery_level: bigint, c02_level: bigint ... 13 more fields]


In [6]:
df.printSchema()
df.show(truncate = false)

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)

+-------------+---------+----+----+-------------+---------+------------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|cn           |device_id|device_name             |humidity|ip             |latitude|lcd   |longitude|scale  |temp|timestamp    |
+-------------+---------+----+----+-------------+---------+------------------------+--------+-------

# Ejercicios página 74

### Detect failing devices with battery levels below a threshold

In [9]:
import org.apache.spark.sql.types.IntegerType

val batteryFilter = df.withColumn("battery_level", col("battery_level").cast(IntegerType) )
                 .where(col("battery_level")<5)

batteryFilter.show(truncate= false)

+-------------+---------+----+----+-----------------+---------+-----------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|cn               |device_id|device_name            |humidity|ip             |latitude|lcd   |longitude|scale  |temp|timestamp    |
+-------------+---------+----+----+-----------------+---------+-----------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|2            |1556     |IT  |ITA |Italy            |3        |device-mac-36TWSKiT    |44      |88.36.5.1      |42.83   |red   |12.83    |Celsius|19  |1458444054120|
|4            |931      |PH  |PHL |Philippines      |5        |therm-stick-5gimpUrBB  |62      |203.82.41.9    |14.58   |green |120.97   |Celsius|25  |1458444054122|
|3            |1210     |US  |USA |United States    |6        |sensor-pad-6al7RTAobR  |51      |204.116.105.67 |35.93   |yellow|-85.46   |Celsius|27  |1458444054122|
|3  

import org.apache.spark.sql.types.IntegerType
batteryFilter: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [battery_level: int, c02_level: bigint ... 13 more fields]


### Identify offending countries with high levels of CO2 emissions

In [13]:
import org.apache.spark.sql.expressions.Window

val w = Window.partitionBy("temp")

val highC02 = df.withColumn("c02_level", col("c02_level").cast(IntegerType))
            .withColumn("mediaC02_temp", mean("c02_level").over(w))
            .where(col("c02_level") > col("mediaC02_temp"))
            .select(col("C02_level"), col("mediaC02_temp"), col("cn"), col("temp"))

highC02.show(truncate= false)

+---------+------------------+-------------+----+
|C02_level|mediaC02_temp     |cn           |temp|
+---------+------------------+-------------+----+
|1470     |1197.9663077897403|United States|26  |
|1399     |1197.9663077897403|United States|26  |
|1553     |1197.9663077897403|Ecuador      |26  |
|1360     |1197.9663077897403|United States|26  |
|1477     |1197.9663077897403|China        |26  |
|1565     |1197.9663077897403|United States|26  |
|1556     |1197.9663077897403|Monaco       |26  |
|1372     |1197.9663077897403|United States|26  |
|1533     |1197.9663077897403|United States|26  |
|1370     |1197.9663077897403|United States|26  |
|1399     |1197.9663077897403|United States|26  |
|1505     |1197.9663077897403|Russia       |26  |
|1545     |1197.9663077897403|United States|26  |
|1525     |1197.9663077897403|Ireland      |26  |
|1261     |1197.9663077897403|Japan        |26  |
|1227     |1197.9663077897403|United States|26  |
|1316     |1197.9663077897403|United States|26  |


import org.apache.spark.sql.expressions.Window
w: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@1240b87d
highC02: org.apache.spark.sql.DataFrame = [C02_level: int, mediaC02_temp: double ... 2 more fields]


### Compute the min and max values for temperature, battery level, C02 and humidity

In [16]:
val minMax = df.withColumn("c02_level", col("c02_level").cast(IntegerType))
            .withColumn("temp", col("temp").cast(IntegerType))
            .withColumn("battery_level", col("battery_level").cast(IntegerType))
            .withColumn("humidity", col("humidity").cast(IntegerType))
            .groupBy("cn")
            .agg(min(col("c02_level")).alias("MinC02"), min(col("battery_level")).alias("minBat"), min(col("temp")).alias("minTemp"), min(col("humidity")).alias("minHum"), 
                 max(col("c02_level")).alias("MaxC02"), max(col("battery_level")).alias("maxBat"), max(col("temp")).alias("maxTemp"), max(col("humidity")).alias("maxHum"))

minMax.show(truncate= false)

+------------------------------+------+------+-------+------+------+------+-------+------+
|cn                            |MinC02|minBat|minTemp|minHum|MaxC02|maxBat|maxTemp|maxHum|
+------------------------------+------+------+-------+------+------+------+-------+------+
|Russia                        |800   |0     |10     |25    |1599  |9     |34     |99    |
|Paraguay                      |836   |0     |10     |27    |1580  |9     |33     |98    |
|Anguilla                      |865   |0     |23     |26    |1398  |9     |34     |83    |
|Macao                         |831   |0     |10     |25    |1592  |9     |34     |98    |
|U.S. Virgin Islands           |819   |0     |10     |25    |1591  |9     |34     |99    |
|Yemen                         |817   |0     |10     |28    |1427  |9     |32     |95    |
|British Indian Ocean Territory|852   |7     |25     |54    |1560  |7     |27     |76    |
|Senegal                       |802   |1     |11     |25    |1589  |9     |34     |94    |

minMax: org.apache.spark.sql.DataFrame = [cn: string, MinC02: int ... 7 more fields]


# 2.- Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por defecto.

In [18]:
// Get the M&M data set filename
val mnmFile = "C:/Users/alejandro.perez/Documents/LearningSparkV2-master/chapter2/scala/data/mnm_dataset.csv"

// Read the file into a Spark DataFrame
val mnmDF = spark.read.format("csv")
 .option("header", "true")
 .option("inferSchema", "true")
 .load(mnmFile)

mnmFile: String = C:/Users/alejandro.perez/Documents/LearningSparkV2-master/chapter2/scala/data/mnm_dataset.csv
mnmDF: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]


In [19]:
mnmDF.show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   WA|Yellow|   20|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
+-----+------+-----+
only showing top 20 rows



In [20]:
mnmDF.printSchema()

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)



# 3.- Cuando se define un schema al definir un campo por ejemplo StructField('Delay', FloatType(), True) ¿qué significa el último parámetro Boolean?

Especifica cuando el campo, en este caso 'Delay' puede tener valores nulos o no, en este caso y por defecto viene asignado el valor True por lo que si puede tener valores nulos.

# 4.- Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?

Los Dataset permiten utilizar funciones de la API de los rdd y están optimizados con Catalyst.

# 5.- Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y guardar los datos en los formatos:

In [22]:
mnmDF.write.format("avro")
      .save("mnmSCALA.avro")

In [23]:
mnmDF.write.format("json")
      .save("mnmSCALA.json")

In [24]:
mnmDF.write.format("csv")
      .save("mnmSCALA.csv")